# ARIMA Forecasting for Prim Eff CBOD5 mg/L
This notebook allows interactive forecasting using the ARIMA model. Adjust the input parameters and observe the forecasted outcomes.

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import plotly.graph_objs as go
from ipywidgets import interact, widgets

## Data Loading and Preprocessing

In [ ]:
# Load the data
forecast_data = pd.read_csv('EaglesPointData.csv', parse_dates=['Date'], encoding='latin1')
forecast_data = forecast_data.sort_values('Date').set_index('Date')

# Extract air flow columns and compute total air flow
relevant_air_flow_columns = [f"SEC Aer Basin {i+1} Oxic Z-{j+1} Air Flow" for i in range(4) for j in range(3)]
forecast_data['Total Air Flow'] = forecast_data[relevant_air_flow_columns].sum(axis=1)

# Split data into training and test sets (80% training, 20% test)
train_size = int(0.8 * len(forecast_data))
train = forecast_data.iloc[:train_size]
test = forecast_data.iloc[train_size:]

# Impute missing values
train_filled = train.fillna(train.mean())
test_filled = test.fillna(train.mean())  # Use mean from training data for imputation in the test set

# Fit the ARIMA model
model_cbod5 = ARIMA(train_filled['Prim Eff CBOD5 mg/L'], order=(5,1,0), exog=train_filled[['Inf water T ¡C', 'Total Air Flow']])
model_cbod5_fit = model_cbod5.fit()

## Interactive Forecasting

In [ ]:
    trace5 = go.Scatter(x=test_filled.index, y=conf_int.iloc[:, 1], mode='lines', name='Upper Bound', fill='tonexty', fillcolor='rgba(255, 0, 0, 0.1)', line=dict(color='pink'))
    layout = go.Layout(title='ARIMA Forecast with Confidence Intervals', xaxis=dict(title='Date'), yaxis=dict(title='Prim Eff CBOD5 mg/L'))
    fig = go.Figure(data=[trace1, trace2, trace3, trace4, trace5], layout=layout)
    fig.show()

# Create interactive widgets for 'Inf water T ¡C' and 'Total Air Flow'
interact(plot_forecast, 
         inf_water=widgets.FloatSlider(min=12.0, max=20.0, step=0.8, value=train_filled['Inf water T ¡C'].mean(), description='Inf water T ¡C'),
         total_air_flow=widgets.FloatSlider(min=2399.25, max=3895.62, step=149.6, value=train_filled['Total Air Flow'].mean(), description='Total Air Flow'))